In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os
import datetime
from dotenv import load_dotenv
from quapopt import ancillary_functions as anf
from quapopt.circuits import backend_utilities as bck_utils
os.makedirs('../temp', exist_ok=True)

load_dotenv();

### Generate a random Hamiltonian instance

* In this example, we generate a random Sherrington-Kirkpatrick Hamiltonian.
* In general, we can generate other classes, or read the instances from file.
* We can also solve the Hamiltonian classicaly, to get the ground state energy and the highest energy. (this, of course, should be done offline, before running the QAOA)



In [3]:
from quapopt.hamiltonians.generators import build_hamiltonian_generator
from quapopt.data_analysis.data_handling import (CoefficientsType,
                                                 CoefficientsDistribution,
                                                 CoefficientsDistributionSpecifier,
                                                 HamiltonianModels)

number_of_qubits = 10
seed_cost_hamiltonian = 1

coefficients_type = CoefficientsType.CONTINUOUS
coefficients_distribution = CoefficientsDistribution.Uniform
coefficients_distribution_properties = {'low': -1, 'high': 1, 'step': 1}
coefficients_distribution_specifier = CoefficientsDistributionSpecifier(CoefficientsType=coefficients_type,
                                                                        CoefficientsDistributionName=coefficients_distribution,
                                                                        CoefficientsDistributionProperties=coefficients_distribution_properties)

# We generate a Hamiltonian instance. In this case it's a random Sherrington-Kirkpatrick Hamiltonian
hamiltonian_model = HamiltonianModels.SherringtonKirkpatrick
localities = (2,)
generator_cost_hamiltonian = build_hamiltonian_generator(hamiltonian_model=hamiltonian_model,
                                                         localities=localities,
                                                         coefficients_distribution_specifier=coefficients_distribution_specifier)

cost_hamiltonian = generator_cost_hamiltonian.generate_instance(number_of_qubits=number_of_qubits,
                                                                seed=seed_cost_hamiltonian,
                                                                read_from_drive_if_present=True)

print("Class description (cost):", cost_hamiltonian.hamiltonian_class_description)
print("Instance description (cost):", cost_hamiltonian.hamiltonian_instance_description)

if cost_hamiltonian.lowest_energy is None:
    print("SOLVING THE HAMILTONIAN CLASSICALLY")
    # if we wish, we can solve the Hamiltonian classically
    cost_hamiltonian.solve_hamiltonian(both_directions=True)

ground_state_energy = cost_hamiltonian.ground_state_energy
highest_energy = cost_hamiltonian.highest_energy

File not found!
FILE NOT FOUND!
Class description (cost): HMN=SK;LOC=(2,);CFD=CT~CON_CDN~UNI_CDP~low~-1_high~1_step~1
Instance description (cost): NOQ=10;HII=1
SOLVING THE HAMILTONIAN CLASSICALLY


### Wrapper for QAOA Sampler
* In the context of QAOA, we can wrap the simulator a bit more so it's a part of more abstract framework useful in further optimization.
* The "QAOARunnerSampler" is an abstract class that can be used to run the QAOA with different backends (here we will be interested mostly in qiskit though).
* The optimization is a bit more general -- a) we can optimize over ansatz for which phase is not the same as cost; b) we can optimize over multiple representations of the Hamiltonian (e.g., permutations or bitflip gauges); note that b) requires classical optimizer that can actually support it.


In [4]:
from quapopt.optimization.QAOA.implementation.QAOARunnerSampler import QAOARunnerSampler
from quapopt.data_analysis.data_handling import LoggingLevel

#Specify some kwargs for logging (this is generally not required, but here we want to log the results in a specific folder)
experiment_set_id = f"{anf.create_random_uuid()}"

logging_level = LoggingLevel.DETAILED

#The logger will create subfolders specified here
logger_kwargs_main = {'experiment_folders_hierarchy': ['SimulationResults',
                                                       'TestingQAOAOptimization'],
                      'experiment_set_id': experiment_set_id,  #Used to group the experiments
                      }

print(logger_kwargs_main)



{'experiment_folders_hierarchy': ['SimulationResults', 'TestingQAOAOptimization'], 'experiment_set_id': '295ef3297ec54d20b7585a672d08fae3'}


In [5]:
sampler_backend = 'Qiskit'

if sampler_backend.lower() == 'qiskit':
    from quapopt.optimization.QAOA import QubitMappingType

    instance_name_ibm = os.getenv('IBM_INSTANCE_NAME')
    account_name_ibm = os.getenv('IBM_ACCOUNT_NAME')
    credentials_path = os.getenv('IBM_CREDENTIALS_PATH')
    #get the provider for the IBM Qiskit backend; we need it to run the SABRE pass manager
    provider_ibm = bck_utils.get_qiskit_provider(instance_ibm=instance_name_ibm,
                                                 account_name=account_name_ibm,
                                                 credentials_path=credentials_path)
    #get a default simulator backend and pass manager for the SABRE pass manager
    #This can be changed if desired
    qiskit_backend, qiskit_pass_manager, pass_manager_kwargs = bck_utils.get_default_qiskit_backend_and_pass_manager(
        provider=provider_ibm,
        backend_name='aer',  #might change to actual backend for simulation for real device
        qubit_mapping_type=QubitMappingType.sabre,
        backend_kwargs=None,  #use defaults
        pass_manager_kwargs=None,  #use defaults
    )

    qiskit_sampler = bck_utils.create_qiskit_sampler(qiskit_backend=qiskit_backend,
                                                     simulation=True,
                                                     qiskit_sampler_options=None,  #use defaults
                                                     session_ibm=None,  #add if desired
                                                     override_to_noiseless_simulation=True,
                                                     #remove default device's noise model
                                                     )




In [6]:
logger_kwargs_sampler = {**logger_kwargs_main,
                         **{'experiment_instance_id': f"IdealSampling{sampler_backend.upper()}"}}

numpy_rng_sampling = np.random.default_rng(seed=0)
#Set up the raw sampler with Hamiltonians as an input
qaoa_sampler = QAOARunnerSampler(
    hamiltonian_representations_cost=[cost_hamiltonian],
    hamiltonian_representations_phase=None,
    store_n_best_results=1,
    store_full_information_in_history=False,
    numpy_rng_sampling=numpy_rng_sampling,
    logger_kwargs=logger_kwargs_sampler,
    logging_level=logging_level
)

#Here we basically run the same thing as before but inside the class;
#so the ansatze are set up for possibly multiple cost Hamiltonians
if sampler_backend.lower() == 'qiskit':
    qaoa_sampler.initialize_backend_qiskit(simulation=True,
                                           qiskit_backend=qiskit_backend,
                                           noiseless_simulation=True,
                                           qiskit_pass_manager=qiskit_pass_manager,
                                           qaoa_depth=1,
                                           qubit_indices_physical=None,
                                           classical_indices=None)
elif sampler_backend.lower() == 'qokit':
    #If we have GPU, we use qokit for demonstration because it's faster than qiskit
    qaoa_sampler.initialize_backend_qokit(qokit_backend='gpu')
else:
    raise NotImplementedError("Only qiskit and qokit are supported")

#print(qaoa_sampler._backends[0].ansatz_circuit)

No existing metadata found for the specified experiment set. 


### Wrapper for optimizers

* In general, we will want to use our QAOA samplers with some optimizers.
* We can run:
1. Optuna -- a hyperparameter optimization framework. It has some nice optimizers and visualization features, but it's not the fastest. Somehow they have huge overheads for long optimizations.
2. Custom grid sampling - we can run a simple grid sampling to get a feel for the landscape.
3. Scipy -- we can also use scipy's optimizers.




In [7]:
from quapopt.optimization.parameter_setting.variational.QAOAOptimizationRunner import QAOAOptimizationRunner
from quapopt.optimization.parameter_setting.variational.scipy_tools.ScipyOptimizerWrapped import ScipyOptimizerWrapped

#we can specify the details of classical optimizer here.
classical_optimizer = ScipyOptimizerWrapped(parameters_bounds=[(-np.pi, np.pi)] * 2,
                                            #argument_names = ['Angles-0', 'Angles-1'],
                                            optimizer_name='COBYLA',
                                            optimizer_kwargs=None,
                                            basinhopping=True,
                                            basinhopping_kwargs={'niter': 1},
                                            starting_point=[0.05] * 2
                                            )

qaoa_optimizer = QAOAOptimizationRunner(qaoa_runner=qaoa_sampler)

In [8]:
#Number of objective function calls
number_of_function_calls = 200
#Number of measurements to estimate the expectation values
number_of_samples = 10 ** 3

best_results_noiseless, optimization_results_noiseless = qaoa_optimizer.run_optimization(qaoa_depth=1,
                                                                                         number_of_function_calls=number_of_function_calls,
                                                                                         number_of_samples=number_of_samples,
                                                                                         classical_optimizer=classical_optimizer,
                                                                                         optimizer_seed=42,
                                                                                         #Note that we can add readout noise here
                                                                                         measurement_noise=None,
                                                                                         verbosity=1,
                                                                                         show_progress_bar=True)



COBYLA:   0%|          | 0/200 [00:00<?, ?it/s]

Return from COBYLA because the trust region radius reaches its lower bound.
Number of function values = 32   Least value of F = -5.26127256154269   Constraint violation = 0.0
The corresponding X is: [ 0.29196801 -0.3464458 ]
The constraint value is:
[-3.43356066 -2.79514685 -2.84962465 -3.48803845]

basinhopping step 0: f -5.26127
Return from COBYLA because the trust region radius reaches its lower bound.
Number of function values = 35   Least value of F = -5.327446242690086   Constraint violation = 0.0
The corresponding X is: [ 0.28315855 -0.37658936]
The constraint value is:
[-3.4247512  -2.7650033  -2.8584341  -3.51818201]

basinhopping step 1: f -5.32745 trial_f -5.32745 accepted True lowest_f -5.32745
found new global minimum on step 1 with function value -5.32745


In [9]:
#note: "best" result here means the one containing the best BITSTRING, not necessary the best expected value
best_result_noiseless = best_results_noiseless[0]
best_energy_bts_noiseless = best_result_noiseless[0]
best_bts_noiseless = best_result_noiseless[1][0]
best_metadata_noiseless = best_result_noiseless[1][-1]

print("BEST ENERGY (BTS, NOISELESS):", best_energy_bts_noiseless)
print("BEST BTS (NOISELESS):", best_bts_noiseless)
print('BEST MEAN ENERGY (NOISELESS):', optimization_results_noiseless.best_value)
best_metadata_noiseless.to_dataframe_main()

BEST ENERGY (BTS, NOISELESS): -11.925219831988215
BEST BTS (NOISELESS): (np.int32(0), np.int32(1), np.int32(1), np.int32(0), np.int32(1), np.int32(1), np.int32(0), np.int32(1), np.int32(0), np.int32(1))
BEST MEAN ENERGY (NOISELESS): -5.327446242690086


,TrialIndex,HamiltonianRepresentationIndex,Angles,EnergyMean,EnergyBest,BitstringBest
0,0,0,"[0.05, 0.05]",0.434445,-11.92522,"[0, 1, 1, 0, 1, 1, 0, 1, 0, 1]"


### Add measurement noise

* We can add classical measurement noise to the simulation. Here we will emulate amplitude damping.

In [10]:
from quapopt.circuits.noise.simulation.ClassicalMeasurementNoiseSampler import ClassicalMeasurementNoiseSampler, \
    MeasurementNoiseType

# Fully asymmetric noise -- equivalent to amplitude damping at the end of the circuit
p_01 = 0.5
p_10 = None
CMNS = ClassicalMeasurementNoiseSampler(noise_type=MeasurementNoiseType.TP_1q_identical,
                                        noise_description={'p_01': p_01,
                                                           'p_10': p_10})

logger_kwargs_sampler_noisy = {**logger_kwargs_main,
                               **{'experiment_instance_id': f"NoisySampling{sampler_backend.upper()}"}}
qaoa_optimizer.qaoa_runner.reinitialize_logger(**logger_kwargs_sampler_noisy)

best_results_noisy, optimization_results_noisy = qaoa_optimizer.run_optimization(qaoa_depth=1,
                                                                                 number_of_function_calls=number_of_function_calls,
                                                                                 number_of_samples=number_of_samples,
                                                                                 classical_optimizer=classical_optimizer,
                                                                                 optimizer_seed=42,
                                                                                 #Note that are adding noise here
                                                                                 measurement_noise=CMNS,
                                                                                 verbosity=1,
                                                                                 show_progress_bar=True
                                                                                 )


COBYLA:   0%|          | 0/200 [00:00<?, ?it/s]

Return from COBYLA because the trust region radius reaches its lower bound.
Number of function values = 23   Least value of F = 0.2280235576480627   Constraint violation = 0.0
The corresponding X is: [0.0134765  0.07861387]
The constraint value is:
[-3.15506916 -3.22020652 -3.12811615 -3.06297878]

basinhopping step 0: f 0.228024
Return from COBYLA because the trust region radius reaches its lower bound.
Number of function values = 32   Least value of F = -1.134992110952735   Constraint violation = 0.0
The corresponding X is: [-0.29288465  0.35614522]
The constraint value is:
[-2.84870801 -3.49773788 -3.4344773  -2.78544743]

basinhopping step 1: f -1.13499 trial_f -1.13499 accepted True lowest_f -1.13499
found new global minimum on step 1 with function value -1.13499


In [11]:
best_result_noisy = best_results_noisy[0]

best_energy_bts_noisy = best_result_noisy[0]
best_bts_noisy = best_result_noisy[1][0]
best_metadata_noisy = best_result_noisy[1][-1]

print("BEST ENERGY (BTS, NOISELESS):", best_energy_bts_noiseless)
print("BEST BTS (NOISELESS):", best_bts_noiseless)
print('BEST MEAN ENERGY (NOISELESS):', optimization_results_noiseless.best_value)
print("____________")
print("BEST ENERGY (BTS, NOISY):", best_energy_bts_noisy)
print("BEST BTS:", best_bts_noisy)
print('BEST MEAN ENERGY (NOISY):', optimization_results_noisy.best_value)

BEST ENERGY (BTS, NOISELESS): -11.925219831988215
BEST BTS (NOISELESS): (np.int32(0), np.int32(1), np.int32(1), np.int32(0), np.int32(1), np.int32(1), np.int32(0), np.int32(1), np.int32(0), np.int32(1))
BEST MEAN ENERGY (NOISELESS): -5.327446242690086
____________
BEST ENERGY (BTS, NOISY): -11.925219831988215
BEST BTS: (np.int32(1), np.int32(0), np.int32(0), np.int32(1), np.int32(0), np.int32(0), np.int32(1), np.int32(0), np.int32(1), np.int32(0))
BEST MEAN ENERGY (NOISY): -1.134992110952735


### Reading data

* In a moment, we will want to visualize data.
* We could take the data directly from the output of optimization, but here we will read it from database to test the logging:
1. We need to set up the logger for the results. (the optimizer does it automatically, but we are pretending we ran the optimization separately and now just reading the results).
2. The input requires: the cost Hamiltonian, and potentially the folder hierarchy and the main table name prefix so it can identify what data we want to read.
3. We can then read the data from the database and visualize it.

In [12]:
from quapopt.data_analysis.data_handling import ResultsLogger
from quapopt.data_analysis.data_handling import (STANDARD_NAMES_DATA_TYPES as SNDT,
                                                 STANDARD_NAMES_VARIABLES as SNV)

results_logger_sampler = ResultsLogger(**logger_kwargs_sampler)

df_noiseless = results_logger_sampler.read_results(data_type=SNDT.OptimizationOverview,
                                                   experiment_instance_ids=['IdealSamplingQISKIT'])
df_noiseless_direct = optimization_results_noiseless.trials_dataframe

#the same logger can be used; instance_ids are used for default writing, but reading is more flexible.
#Both writing and reading filter by experiment_set_id by default
df_noisy = results_logger_sampler.read_results(data_type=SNDT.OptimizationOverview,
                                               experiment_instance_ids=['NoisySamplingQISKIT'])
df_noisy_direct = optimization_results_noisy.trials_dataframe


In [13]:
df_noiseless

,TrialIndex,HamiltonianRepresentationIndex,Angles,EnergyMean,EnergyBest,BitstringBest,ExperimentInstanceID
0,0,0,"[0.05, 0.05]",0.434445,-11.925220,"[0, 1, 1, 0, 1, 1, 0, 1, 0, 1]",IdealSamplingQISKIT
1,1,0,"[0.15000000000000002, 0.05]",0.818728,-11.925220,"[1, 0, 0, 1, 0, 0, 1, 0, 1, 0]",IdealSamplingQISKIT
2,2,0,"[0.05, 0.15000000000000002]",0.858228,-11.925220,"[1, 0, 0, 1, 0, 0, 1, 0, 1, 0]",IdealSamplingQISKIT
3,3,0,"[-0.017173958066904027, -0.024078737554212]",-0.043542,-10.400989,"[1, 0, 0, 1, 0, 0, 0, 0, 1, 0]",IdealSamplingQISKIT
4,4,0,"[-0.175464311581699, -0.14632506258756928]",2.543370,-8.162780,"[0, 1, 1, 0, 1, 1, 1, 1, 0, 0]",IdealSamplingQISKIT
...,...,...,...,...,...,...,...
62,62,0,"[0.2831459036188167, -0.37650418019395204]",-5.322294,-11.925220,"[0, 1, 1, 0, 1, 1, 0, 1, 0, 1]",IdealSamplingQISKIT
63,63,0,"[0.28299903177374763, -0.3764881409387708]",-5.318779,-11.925220,"[0, 1, 1, 0, 1, 1, 0, 1, 0, 1]",IdealSamplingQISKIT
64,64,0,"[0.2831078229013156, -0.3764623632243164]",-5.322641,-11.925220,"[1, 0, 0, 1, 0, 0, 1, 0, 1, 0]",IdealSamplingQISKIT
65,65,0,"[0.2831585490130242, -0.37658935818297234]",-5.327446,-11.925220,"[0, 1, 1, 0, 1, 1, 0, 1, 0, 1]",IdealSamplingQISKIT


In [14]:
df_noiseless_direct

,FunctionValue,ARG-0,ARG-1,TrialIndex
0,0.434445,0.050000,0.050000,0
1,0.818728,0.150000,0.050000,1
2,0.858228,0.050000,0.150000,2
3,-0.043542,-0.017174,-0.024079,3
4,2.543370,-0.175464,-0.146325,4
...,...,...,...,...
62,-5.322294,0.283146,-0.376504,62
63,-5.318779,0.282999,-0.376488,63
64,-5.322641,0.283108,-0.376462,64
65,-5.327446,0.283159,-0.376589,65


In [15]:
df_noisy

,TrialIndex,HamiltonianRepresentationIndex,Angles,EnergyMean,EnergyBest,BitstringBest,ExperimentInstanceID
67,0,0,"[0.05, 0.05]",0.302911,-11.925220,"[1, 0, 0, 1, 0, 0, 1, 0, 1, 0]",NoisySamplingQISKIT
68,1,0,"[0.15000000000000002, 0.05]",0.562891,-10.400989,"[1, 0, 0, 1, 0, 0, 0, 0, 1, 0]",NoisySamplingQISKIT
69,2,0,"[0.05, 0.15000000000000002]",0.689012,-8.237507,"[1, 0, 0, 1, 0, 1, 0, 0, 1, 0]",NoisySamplingQISKIT
70,3,0,"[-0.005853068062424464, -0.03294838628939192]",0.353256,-10.400989,"[1, 0, 0, 1, 0, 0, 0, 0, 1, 0]",NoisySamplingQISKIT
71,4,0,"[0.012960839571947519, 0.08358720879717443]",0.282384,-10.400989,"[1, 0, 0, 1, 0, 0, 0, 0, 1, 0]",NoisySamplingQISKIT
72,5,0,"[0.012765828092611831, 0.13358682850095746]",0.340664,-11.925220,"[0, 1, 1, 0, 1, 1, 0, 1, 0, 1]",NoisySamplingQISKIT
73,6,0,"[0.00046093464600176215, 0.08353845592734051]",0.355734,-10.400989,"[1, 0, 0, 1, 0, 0, 0, 0, 1, 0]",NoisySamplingQISKIT
74,7,0,"[0.03750058170199547, 0.07881216364652403]",0.563249,-11.925220,"[1, 0, 0, 1, 0, 0, 1, 0, 1, 0]",NoisySamplingQISKIT
75,8,0,"[0.0030141638116315175, 0.08255587891825975]",0.426879,-10.400989,"[1, 0, 0, 1, 0, 0, 0, 0, 1, 0]",NoisySamplingQISKIT
76,9,0,"[0.013476504511404861, 0.07861387091701644]",0.228024,-10.400989,"[1, 0, 0, 1, 0, 0, 0, 0, 1, 0]",NoisySamplingQISKIT


In [16]:
df_noisy_direct

,FunctionValue,ARG-0,ARG-1,TrialIndex
0,0.302911,0.050000,0.050000,0
1,0.562891,0.150000,0.050000,1
2,0.689012,0.050000,0.150000,2
3,0.353256,-0.005853,-0.032948,3
4,0.282384,0.012961,0.083587,4
5,0.340664,0.012766,0.133587,5
6,0.355734,0.000461,0.083538,6
7,0.563249,0.037501,0.078812,7
8,0.426879,0.003014,0.082556,8
9,0.228024,0.013477,0.078614,9


### Visualization

* After running optimization, we can visualize it using various methods with plotly.


In [17]:
import plotly
from quapopt.data_analysis.visualization import optimization_visualization as opt_vis

plotly.io.templates.default = "plotly"
plotly.offline.init_notebook_mode(connected=True)

x_name = f"{SNV.Angles.id_long}-0"
y_name = f"{SNV.Angles.id_long}-1"
fom_name = f"{SNV.EnergyMean.id_long}"

for _df in [df_noiseless, df_noisy]:
    _df[x_name] = _df['Angles'].apply(lambda x: x[0])
    _df[y_name] = _df['Angles'].apply(lambda x: x[1])

In [18]:
trajectory_fig = opt_vis.plot_multiple_heatmaps(dataframes=[df_noiseless, df_noisy],
                                                x_name=x_name,
                                                y_name=y_name,
                                                fom_name=fom_name,
                                                bounds_x=(-np.pi, np.pi),
                                                bounds_y=(-np.pi, np.pi),
                                                add_trajectories=True,
                                                titles=['Ideal', 'Noisy'],
                                                global_normalization=True,
                                                colormap_name='Viridis',
                                                minimization=True)

plotly.offline.plot(trajectory_fig,
                    filename='../temp/trajectories_plot.html')
# trajectory_fig.show()


'../temp/trajectories_plot.html'

### Grid sampling
* We can also run a simple grid sampling to get a feel for the landscape and to compare it with the optimization results.
* Here we run bruteforce grid sampling that is not scalable for large dimensions
* We will use it to plot the background grid of points for the trajectories of the TPE optimizer.
* Note that this works only for p=1, otherwise it's hard to visualize

In [19]:
from quapopt.optimization.parameter_setting.non_adaptive_optimization.SimpleGridOptimizer import SimpleGridOptimizer
from quapopt.optimization.parameter_setting import ParametersBoundType as PBT
from quapopt.optimization.QAOA.simulation.QAOARunnerExpValues import QAOARunnerExpValues

grid_size_total = 10 ** 4

grid_sampler = SimpleGridOptimizer(parameter_bounds=[(PBT.RANGE, (-np.pi, np.pi)),
                                                     (PBT.RANGE, (-np.pi, np.pi))
                                                     ],
                                   max_trials=grid_size_total)

logger_kwargs_grid = {**logger_kwargs_main,
                      **{'experiment_instance_id': 'IdealExpValuesGrid'}}

#runner that calculates expected values for p=1 QAOA
qaoa_exp_values_simulator = QAOARunnerExpValues(
    hamiltonian_representations_cost=[cost_hamiltonian],
    store_full_information_in_history=False,
    simulator_name=None,
    logger_kwargs=logger_kwargs_grid,
    logging_level=None)

qaoa_optimizer_exp_values = QAOAOptimizationRunner(qaoa_runner=qaoa_exp_values_simulator)


In [20]:
#noiseless grid
best_result_grid_noiseless, opt_results_grid_noiseless = qaoa_optimizer_exp_values.run_optimization(
    qaoa_depth=1,
    number_of_function_calls=grid_size_total,
    classical_optimizer=grid_sampler,
    measurement_noise=None,
    memory_intensive=True,
    show_progress_bar=True,
    verbosity=1,
    # analytical_betas=True
)


  0%|          | 0/10000 [00:00<?, ?it/s]

New minimum found at trial: 0 with function value: inf -> 1.8970323264925344e-16
New minimum found at trial: 12 with function value: 1.8970323264925344e-16 -> -0.07159083503163204
New minimum found at trial: 13 with function value: -0.07159083503163204 -> -0.16915375259296894
New minimum found at trial: 14 with function value: -0.16915375259296894 -> -0.25935579529291497
New minimum found at trial: 15 with function value: -0.25935579529291497 -> -0.33641477792100044
New minimum found at trial: 16 with function value: -0.33641477792100044 -> -0.39539101955261463
New minimum found at trial: 17 with function value: -0.39539101955261463 -> -0.43250398992564654
New minimum found at trial: 18 with function value: -0.43250398992564654 -> -0.4453746512491074
New minimum found at trial: 3803 with function value: -0.4453746512491074 -> -0.521279794070765
New minimum found at trial: 3804 with function value: -0.521279794070765 -> -0.6224219729369312
New minimum found at trial: 3805 with function 

In [21]:
#noisy grid
best_result_grid_noisy, opt_results_grid_noisy = qaoa_optimizer_exp_values.run_optimization(qaoa_depth=1,
                                                                                            number_of_function_calls=grid_size_total,
                                                                                            classical_optimizer=grid_sampler,
                                                                                            memory_intensive=True,
                                                                                            verbosity=1,
                                                                                            measurement_noise=CMNS,
                                                                                            show_progress_bar=True
                                                                                            )

  0%|          | 0/10000 [00:00<?, ?it/s]

New minimum found at trial: 0 with function value: inf -> 0.37237026472161294
New minimum found at trial: 12 with function value: 0.37237026472161294 -> 0.35447255575193043
New minimum found at trial: 13 with function value: 0.35447255575193043 -> 0.33008182608653114
New minimum found at trial: 14 with function value: 0.33008182608653114 -> 0.3075313151575497
New minimum found at trial: 15 with function value: 0.3075313151575497 -> 0.2882665692838853
New minimum found at trial: 16 with function value: 0.2882665692838853 -> 0.273522508710578
New minimum found at trial: 17 with function value: 0.273522508710578 -> 0.26424426601375833
New minimum found at trial: 18 with function value: 0.26424426601375833 -> 0.26102660064781213
New minimum found at trial: 3803 with function value: 0.26102660064781213 -> 0.24205031585210757
New minimum found at trial: 3804 with function value: 0.24205031585210757 -> 0.2167647711156983
New minimum found at trial: 3805 with function value: 0.2167647711156983

In [22]:
#this can made it a bit slower than just opening in browser, but it allows to show the plots in Jupyter cells
show_plots_in_jupyter_cell = False

#TODO(FBM): this should be unnecessary, refactor
names_map = {f"ARG-0": f"{SNV.Angles.id_long}-0",
             f"ARG-1": f"{SNV.Angles.id_long}-1",
             f"FunctionValue": f"{SNV.EnergyMean.id_long}"}

df_noiseless_grid = opt_results_grid_noiseless.trials_dataframe
df_noiseless_grid.rename(columns=names_map, inplace=True)

df_noisy_grid = opt_results_grid_noisy.trials_dataframe
df_noisy_grid.rename(columns=names_map, inplace=True)

for plot_in_3d in [False, True]:
    heatmap_grid_noiseless, scatter_grid_noiseless = opt_vis.get_interpolated_heatmap(df=df_noiseless_grid,
                                                                                      x_name=x_name,
                                                                                      y_name=y_name,
                                                                                      fom_name=fom_name,
                                                                                      bounds_x=(-np.pi, np.pi),
                                                                                      bounds_y=(-np.pi, np.pi),
                                                                                      in_3d=plot_in_3d
                                                                                      )
    heatmap_grid_noisy, scatter_grid_noisy = opt_vis.get_interpolated_heatmap(df=df_noisy_grid,
                                                                              x_name=x_name,
                                                                              y_name=y_name,
                                                                              fom_name=fom_name,
                                                                              bounds_x=(-np.pi, np.pi),
                                                                              bounds_y=(-np.pi, np.pi),
                                                                              in_3d=plot_in_3d
                                                                              )

    trajectory_fig2 = opt_vis.plot_multiple_heatmaps(dataframes=[df_noiseless, df_noisy],
                                                     x_name=x_name,
                                                     y_name=y_name,
                                                     fom_name=fom_name,
                                                     bounds_x=(-np.pi, np.pi),
                                                     bounds_y=(-np.pi, np.pi),
                                                     heatmaps_input=[heatmap_grid_noiseless, heatmap_grid_noisy],
                                                     scatter_inputs=[scatter_grid_noiseless, scatter_grid_noisy],
                                                     add_trajectories=True,
                                                     titles=['Ideal', 'Noisy'],
                                                     suffixes=['(Ideal)', '(Noisy)'],
                                                     global_normalization=False,
                                                     colormap_name='Viridis',
                                                     minimization=True,
                                                     in_3d=plot_in_3d)

    plotly.offline.plot(trajectory_fig2,
                        filename=f'../temp/trajectories_grid_plot_3d-{plot_in_3d}.html')
    if not show_plots_in_jupyter_cell:
        continue
    trajectory_fig2.show()
